<a href="https://colab.research.google.com/github/fatemehjafari77/cleaning-code/blob/main/Us%20acciddent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # install kaggle

In [1]:
! pip install kaggle --quiet

#  make folder, copy data kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# paste kaggle address

In [3]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

Dataset URL: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
License(s): CC-BY-NC-SA-4.0
 96% 630M/653M [00:07<00:00, 39.2MB/s]
100% 653M/653M [00:07<00:00, 89.9MB/s]


# unzip file

In [4]:
!unzip us-accidents.zip

Archive:  us-accidents.zip
  inflating: US_Accidents_March23.csv  


# make libraries suitable for GPU

In [5]:
!git clone http://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 597 (delta 128), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (597/597), 196.59 KiB | 9.36 MiB/s, done.
Resolving deltas: 100% (302/302), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.13 environment at: /usr
Resolved 175 packages in 1.39s
 Downloaded cuproj-cu12
 Downloaded rmm-cu12
 Downloaded shapely
 Downloaded datashader
 Downloaded dask
 Downloaded pylibcugraph-cu12
 Downloaded ucx-py-cu12
 Downloaded libkvikio-cu12
 Downloaded cugraph-cu12
 Downloaded cuspatial-cu12
 Downloaded librmm-cu12
 Downloaded bokeh
 Downloaded cudf-cu12
 Downloaded cucim-cu12
 Downloaded cuml-cu12
 Downloaded libcuspatial-cu12
 Downloaded pylibcudf-cu12
 Downloaded nvidia-nvcomp-cu12
 Downloaded libraft-cu12
 Downloaded libcudf-cu12
 Downloaded raft-dask-cu12
 Downloaded l

# import libreries

In [6]:
import cudf
import cupy as cp
df = cudf.read_csv("/content/US_Accidents_March23.csv")
import numpy as np

# use info for recive some information about data

In [7]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype
---  ------                 -----
 0   ID                     object
 1   Source                 object
 2   Severity               int64
 3   Start_Time             object
 4   End_Time               object
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object
 11  Street                 object
 12  City                   object
 13  County                 object
 14  State                  object
 15  Zipcode                object
 16  Country                object
 17  Timezone               object
 18  Airport_Code           object
 19  Weather_Timestamp      object
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)            float64
 23  

# cleaning data
### at first we deleted extra columns

In [8]:
#we dont need this columns for analyze so deleted them and we reduce the volume
df.drop(columns=["ID","Country","Astronomical_Twilight","Nautical_Twilight","Civil_Twilight","Sunrise_Sunset","Wind_Speed(mph)"
,"Wind_Direction","Humidity(%)","Wind_Chill(F)","Temperature(F)","Zipcode","Station","County","Source"],inplace=True)

# type casting

In [9]:
#change the type for some columns beacuse they wastes memory
df["Severity"] = df["Severity"].astype("uint8")
df[["Start_Lat","Start_Lng","End_Lat","End_Lng","Distance(mi)","Visibility(mi)","Precipitation(in)"]] = df[["Start_Lat","Start_Lng","End_Lat","End_Lng","Distance(mi)","Visibility(mi)","Precipitation(in)"]].astype("float32")

In [12]:
#boolean change to int,its better for working with data
df[["Bump","Crossing","Give_Way","Junction","No_Exit","Railway","Roundabout","Stop","Traffic_Calming","Traffic_Signal","Turning_Loop"]] = df[["Bump","Crossing","Give_Way","Junction","No_Exit","Railway","Roundabout","Stop","Traffic_Calming","Traffic_Signal","Turning_Loop"]].astype("uint8")

# fill some nan date and clean some of them

In [ ]:
# NAN data is string so we need to change np.nan
df["End_Lng"] = df["End_Lng"].astype(str)
df["End_Lat"] = df["End_Lat"].astype(str)
df["End_Lng"] = df["End_Lng"].replace("NA", cudf.NA)
df["End_Lat"] = df["End_Lat"].replace("NA", cudf.NA)
df["End_Lng"] = df["End_Lng"].astype("float32")
df["End_Lat"] = df["End_Lat"].astype("float32")
df["End_Lng"] = df["End_Lng"].fillna(1)
df["End_Lat"] = df["End_Lat"].fillna(1)

# check the NANE Data

In [13]:
print(df.isna().sum())

Severity                   0
Start_Time                 0
End_Time                   0
Start_Lat                  0
Start_Lng                  0
End_Lat              3402762
End_Lng              3402762
Distance(mi)               0
Description                5
Street                 10869
City                     253
State                      0
Timezone                7808
Airport_Code           22635
Weather_Timestamp     120228
Pressure(in)          140679
Visibility(mi)        177098
Precipitation(in)    2203586
Weather_Condition     173459
Amenity                    0
Bump                       0
Crossing                   0
Give_Way                   0
Junction                   0
No_Exit                    0
Railway                    0
Roundabout                 0
Stop                       0
Traffic_Calming            0
Traffic_Signal             0
Turning_Loop               0
dtype: int64


# find out-lier data

In [28]:
# we want to fin out-lier related to Start_Lat and Start_Lng
### start  with find mean and std for Start_Lat
mean1 = df["Start_Lat"].mean()
std1 = df["Start_Lat"].std()
upper_bond1 = mean1 + 2 * std1
lower_bond1 = mean1 - 2 * std1
outlier1= df["Start_Lat"][(df["Start_Lat"] > upper_bond1) | (df["Start_Lat"] < lower_bond1)].reset_index(drop=True)
print(outlier1.shape[0])

402966


In [29]:
### start  with find mean and std for Start_Lng
mean2 = df["Start_Lng"].mean()
std2 = df["Start_Lng"].std()
upper_bond2 = mean2 + 1.5 * std2
lower_bond2 = mean2 - 1.5 * std2
outlier2 = df["Start_Lng"][(df["Start_Lng"] > upper_bond2) | (df["Start_Lng"] < lower_bond2)].reset_index(drop=True)
print(outlier2.shape[0])

874356


# make outlier file

In [32]:

outlier1 = outlier1.reset_index()
outlier2 = outlier1.reset_index()
outlier1.to_csv("outlier_Lat.csv", index=False)
outlier2.to_csv("outlier_Lng.csv", index=False)